In [1]:
import pandas as pd

# Load the CSV file
file_path = 'kaggleDBQA.csv'
data = pd.read_csv(file_path)

# Check the first few rows to understand its structure
data.head()


,Unnamed: 0,db_id,query,query_toks,question,question_toks,sql
0,0,GeoNuclearData,SELECT Country FROM nuclear_power_plants WHERE...,"['SELECT', 'Country', 'FROM', 'nuclear_power_p...",What are the top 10 countries with most number...,"['What', 'are', 'the', 'top', '10', 'countries...","{'from': {'table_units': [['table_unit', 0]], ..."
1,1,GeoNuclearData,"SELECT Name, Country FROM nuclear_power_plants...","['SELECT', 'Name,', 'Country', 'FROM', 'nuclea...",What are the planed nuclear power plants and t...,"['What', 'are', 'the', 'planed', 'nuclear', 'p...","{'from': {'table_units': [['table_unit', 0]], ..."
2,2,GeoNuclearData,SELECT ReactorModel FROM nuclear_power_plants ...,"['SELECT', 'ReactorModel', 'FROM', 'nuclear_po...",What’s the most used nuclear reactor model?,"['What’s', 'the', 'most', 'used', 'nuclear', '...","{'from': {'table_units': [['table_unit', 0]], ..."
3,3,GeoNuclearData,SELECT Country FROM nuclear_power_plants WHERE...,"['SELECT', 'Country', 'FROM', 'nuclear_power_p...",Which country is Kaiga-4 built in?,"['Which', 'country', 'is', 'Kaiga-4', 'built',...","{'from': {'table_units': [['table_unit', 0]], ..."
4,4,GeoNuclearData,SELECT count(*) FROM nuclear_power_plants WHER...,"['SELECT', 'count(*)', 'FROM', 'nuclear_power_...",How many PHWR are there today?,"['How', 'many', 'PHWR', 'are', 'there', 'today?']","{'from': {'table_units': [['table_unit', 0]], ..."


In [2]:
import re

# Function to prepend db_id to the table name in the query
def transform_query(original_query, db_id):
    # Regular expression to find the table name following "FROM" or "JOIN" clauses
    table_pattern = re.compile(r'\bFROM\b\s+(\w+)|\bJOIN\b\s+(\w+)', re.IGNORECASE)

    # Function to replace the table name with "db_id.table_name"
    def repl_func(match):
        table_name = match.group(1) or match.group(2)  # Get the matched table name
        return f"{match.group(0).split()[0]} {db_id}.{table_name}"  # Prepend db_id

    # Replace all occurrences of table names in the query
    transformed_query = table_pattern.sub(repl_func, original_query)
    
    # Making sure the transformation does not affect case of SQL keywords
    transformed_query = re.sub(r'\b(select|from|where|group by|order by|limit)\b', lambda match: match.group(0).upper(), transformed_query)
    
    return transformed_query

# Apply the transformation to each row in the DataFrame
data['db2_query'] = data.apply(lambda row: transform_query(row['query'], row['db_id']), axis=1)

# Show the first few rows of the updated DataFrame to verify the transformation
data.head()


,Unnamed: 0,db_id,query,query_toks,question,question_toks,sql,db2_query
0,0,GeoNuclearData,SELECT Country FROM nuclear_power_plants WHERE...,"['SELECT', 'Country', 'FROM', 'nuclear_power_p...",What are the top 10 countries with most number...,"['What', 'are', 'the', 'top', '10', 'countries...","{'from': {'table_units': [['table_unit', 0]], ...",SELECT Country FROM GeoNuclearData.nuclear_pow...
1,1,GeoNuclearData,"SELECT Name, Country FROM nuclear_power_plants...","['SELECT', 'Name,', 'Country', 'FROM', 'nuclea...",What are the planed nuclear power plants and t...,"['What', 'are', 'the', 'planed', 'nuclear', 'p...","{'from': {'table_units': [['table_unit', 0]], ...","SELECT Name, Country FROM GeoNuclearData.nucle..."
2,2,GeoNuclearData,SELECT ReactorModel FROM nuclear_power_plants ...,"['SELECT', 'ReactorModel', 'FROM', 'nuclear_po...",What’s the most used nuclear reactor model?,"['What’s', 'the', 'most', 'used', 'nuclear', '...","{'from': {'table_units': [['table_unit', 0]], ...",SELECT ReactorModel FROM GeoNuclearData.nuclea...
3,3,GeoNuclearData,SELECT Country FROM nuclear_power_plants WHERE...,"['SELECT', 'Country', 'FROM', 'nuclear_power_p...",Which country is Kaiga-4 built in?,"['Which', 'country', 'is', 'Kaiga-4', 'built',...","{'from': {'table_units': [['table_unit', 0]], ...",SELECT Country FROM GeoNuclearData.nuclear_pow...
4,4,GeoNuclearData,SELECT count(*) FROM nuclear_power_plants WHER...,"['SELECT', 'count(*)', 'FROM', 'nuclear_power_...",How many PHWR are there today?,"['How', 'many', 'PHWR', 'are', 'there', 'today?']","{'from': {'table_units': [['table_unit', 0]], ...",SELECT count(*) FROM GeoNuclearData.nuclear_po...


In [5]:
data_sample = data[data['db_id']=="TheHistoryofBaseball"]
data_sample.head()

,Unnamed: 0,db_id,query,query_toks,question,question_toks,sql,db2_query
274,0,TheHistoryofBaseball,SELECT T2.birth_state FROM player AS T2 JOIN h...,"['SELECT', 'T2.birth_state', 'FROM', 'player',...",Which states have produced the largest number ...,"['Which', 'states', 'have', 'produced', 'the',...","{'from': {'table_units': [['table_unit', 4], [...",SELECT T2.birth_state FROM TheHistoryofBasebal...
275,1,TheHistoryofBaseball,SELECT league_id FROM salary GROUP BY league_i...,"['SELECT', 'league_id', 'FROM', 'salary', 'GRO...",What is the top league that pays the most to t...,"['What', 'is', 'the', 'top', 'league', 'that',...","{'from': {'table_units': [['table_unit', 3]], ...",SELECT league_id FROM TheHistoryofBaseball.sal...
276,2,TheHistoryofBaseball,SELECT league_id FROM salary GROUP BY league_i...,"['SELECT', 'league_id', 'FROM', 'salary', 'GRO...",Which leage have the largest difference in pay...,"['Which', 'leage', 'have', 'the', 'largest', '...","{'from': {'table_units': [['table_unit', 3]], ...",SELECT league_id FROM TheHistoryofBaseball.sal...
277,3,TheHistoryofBaseball,SELECT avg(T2.salary) FROM salary as T2 JOIN h...,"['SELECT', 'avg(T2.salary)', 'FROM', 'salary',...",What is the average pay for players inducted i...,"['What', 'is', 'the', 'average', 'pay', 'for',...","{'from': {'table_units': [['table_unit', 3], [...",SELECT avg(T2.salary) FROM TheHistoryofBasebal...
278,4,TheHistoryofBaseball,SELECT avg(T2.salary) FROM salary as T2 JOIN h...,"['SELECT', 'avg(T2.salary)', 'FROM', 'salary',...",What is the average pay for players not induct...,"['What', 'is', 'the', 'average', 'pay', 'for',...","{'from': {'table_units': [['table_unit', 3], [...",SELECT avg(T2.salary) FROM TheHistoryofBasebal...


In [8]:
data_sample['db2_query'][348]

'SELECT count(*) FROM (SELECT player_id FROM TheHistoryofBaseball.player_award GROUP BY player_id HAVING count(*) > 10)'